In [1]:
import os
import subprocess
import pandas as pd 
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')
outdir = 'results/refs/ensembl/'
os.makedirs(outdir, exist_ok=True)

fn = os.path.join(outdir, 'gencode.v19.annotation.gtf.gz')
if not os.path.exists(fn):
    url = 'https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_19/gencode.v19.annotation.gtf.gz'
    cmd = 'wget -O {} {}'.format(fn, url)
    output = subprocess.check_output(cmd, shell=True)

In [2]:
gtf = pd.read_table(fn, header=None, skiprows=5)

In [22]:
snp = gtf.loc[gtf[8].str.match('.*RP11-632C17__A.1'), :]

In [26]:
# extract the main columns
parsed = gtf.iloc[:, [0,3,4,8,6,2]]

In [27]:
# get the gene id and gene name
parsed['gene_id'] = parsed[8].str.extract('gene_id "(ENSG[0-9.]+)"')
parsed['gene_name'] = parsed[8].str.extract('gene_name "([A-Za-z0-9-\._]+)"')

# final data cleanup
parsed = parsed.iloc[:, [0,1,2,4,5,6,7]]

/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
# use the gene ID as the name when the name is not 
# available
new_genenames = []
for ID, name in parsed.values[:, 5:7]:
    if type(name) == str:
        new_genenames.append(name)
    else:
        new_genenames.append(ID)
parsed.loc[:, 'gene_name'] = new_genenames

In [29]:
# save the final output
bed = os.path.join(outdir, 'gencode.v19.annotation.bed')
parsed.to_csv(bed, sep='\t', header=None, index=False)